In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/weatheraus/weatherAUS.csv


Günlük nem oranı hesaplamak için gün içindeki nem oranlarının ortalamasını alan fonksiyon

In [2]:
def humidityAvg(humidityAM, humidityPM):
    humidity = []
    for i in range(len(humidityPM)):
        humidity.append((humidityAM[i]+humidityPM[i])/2)
    humidity = np.array(humidity)
    return humidity

In [3]:
datafile = pd.read_csv("../input/weatheraus/weatherAUS.csv")

FileNotFoundError: [Errno 2] No such file or directory: '..\\input\\weatheraus\\weatherAUS.csv'

**Veri Analizi**

In [ ]:
datafile.head()

In [ ]:
datafile.shape

In [ ]:
datafile.dtypes

humidityAVG fonk çağrılması

In [ ]:
humidity = humidityAvg(datafile["Humidity9am"].tolist(),datafile["Humidity3pm"].tolist())
datafile["humAvg"] = humidity

In [ ]:
# Sınıflandırma için kullanılmayacak niteliklerin veri setinden çıkarılması
datafile = datafile.drop(["Sunshine","Evaporation","WindGustDir","WindGustSpeed","WindDir9am","WindDir3pm","WindSpeed9am","WindSpeed3pm","Humidity9am","Humidity3pm","Pressure9am","Pressure3pm","Cloud9am","Cloud3pm","RainTomorrow"], axis=1)


In [ ]:
datafile.describe()

In [ ]:
sns.boxplot(x = 'RainToday', y = 'humAvg', data = datafile[datafile["Location"] == "Albury"]).set_title("Albury nem-yağış")

In [ ]:
sns.barplot(x = 'Location', y = 'humAvg', data = datafile[np.logical_or(datafile["Location"] == "Albury", datafile["Location"] == "Sydney")], hue = 'RainToday').set_title("Albury and Sydney nem-yağış")


**Data pre-proccessing**

In [ ]:
datafile.isnull().sum() # Eksik veri kontrolü

In [ ]:
rainTodayCount = datafile['RainToday'].value_counts() # Bir nitelik için En sık geçen sınıfı bulma

In [ ]:
rainTodayCount = rainTodayCount.fillna("no") # Eksik verileri sabit bir değer ile tamamlama

In [ ]:
from sklearn.preprocessing import LabelEncoder # Kategorik verileri donusturme

In [ ]:
LE = LabelEncoder()

datafile['RainTodayMap'] = datafile.RainToday.map({'NA':'no'})

datafile['Location_Cat'] = LE.fit_transform(datafile['Location'])
datafile['RainToday_Cat'] = LE.fit_transform(datafile['RainTodayMap'])

In [ ]:
datafile.drop(["Date","Location","RainTodayMap","RainToday"], axis=1, inplace=True)

In [ ]:
from sklearn.impute import KNNImputer  # Eksik verileri KNN algoritması ile tamamlama
imputer = KNNImputer(missing_values=np.nan, n_neighbors=5)

datafile[:] = imputer.fit_transform(datafile)

df = pd.DataFrame(datafile)

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer # Sürekli Verileri düzleştirme

In [ ]:
Xbin = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile') # Nem oranini 10 degerden olusan kategorik veriye donusturme
df["humAvg"] = Xbin.fit_transform(df[["humAvg"]])

In [ ]:
df ## Makine Öğrenimi Algoritmasının uygulanacağı veri seti

**Prepare train & test set**

In [ ]:
X = df.loc[:,:"Location_Cat"]  # Sadece niteliklerden olusan Veriseti
y = df.loc[:,"RainToday_Cat"] # Sinif etiketi

In [ ]:
#Veri seti %70 train, %30 test olarak bölmek
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

**KNN ALGORITHM**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # KNN algoritması ---------------------------------------------------
knn = KNeighborsClassifier(n_neighbors=5, metric='minkowski')

knn.fit(X_train, y_train.ravel())

In [ ]:
# Modelin test verisi ile sınıflandırma yapması
result = knn.predict(X_test)
result

In [ ]:
# Modelin Sınıflandırma tahmin başarısının gösterilmesi
from sklearn.metrics import accuracy_score #basari orani
accuracy = accuracy_score(y_test, result)
accuracy

> KNN Modelinin kaydedilip tekrar kullanılması

In [ ]:
import pickle
with open('knnmodel.pkl', 'wb') as f:
    pickle.dump(knn, f)
knn_2 = pickle.load(open('knnmodel.pkl', 'rb'))
result2 = knn_2.predict(X_test)
result2

In [ ]:
accuracy2 = accuracy_score(y_test, result2)
accuracy2

> KNN Karmaşıklık matrisi

In [ ]:
from sklearn.metrics import confusion_matrix #karmasiklik matrisi
cm = confusion_matrix(y_test, result2, labels=[0,1])
cm

**Decision Tree ALGORITHM**

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
dtc = DecisionTreeClassifier(random_state=42)

dtc.fit(X_train, y_train)

In [ ]:
result = dtc.predict(X_test)
result

In [ ]:
accuracy = accuracy_score(y_test, result) #basari orani
accuracy

> Decision Tree Modelinin kaydedilip tekrar kullanılması

In [ ]:
import pickle
with open('dtcmodel.pkl', 'wb') as f:
    pickle.dump(knn, f)
dtc_2 = pickle.load(open('dtcmodel.pkl', 'rb'))
result2 = knn_2.predict(X_test)
result2

In [ ]:
accuracy2 = accuracy_score(y_test, result2)
accuracy2

*Decision Tree karmaşıklık matrisi*

In [ ]:
cm = confusion_matrix(y_test, result2, labels=[0,1])
cm

**NaiveBayes ALGORITHM(GaussianNB)**

In [ ]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB()

gnb.fit(X_train, y_train.ravel())

In [ ]:
result = gnb.predict(X_test)
result

In [ ]:
accuracy = accuracy_score(y_test, result) #basari orani
accuracy

*NaiveBayes Modelinin kaydedilip tekrar kullanılması*

In [ ]:
import pickle
with open('gnbmodel.pkl', 'wb') as f:
    pickle.dump(knn, f)
gnb_2 = pickle.load(open('gnbmodel.pkl', 'rb'))
result2 = knn_2.predict(X_test)
result2

In [ ]:
accuracy2 = accuracy_score(y_test, result2)
accuracy2

*NaiveBayes karmaşıklık matrisi*

In [ ]:
cm = confusion_matrix(y_test, result2, labels=[0,1]) #karmasiklik matrisi
cm